# Comprehensive Guide to Image Filtering Using Convolution in OpenCV
---
---
This notebook provides an in-depth exploration of image filtering techniques using convolution in OpenCV. It is inspired by and expands upon the concepts from the website: https://learnopencv.com/image-filtering-using-convolution-in-opencv/#intro-convo-kernels.

We cover fundamental to advanced topics, including theoretical explanations, mathematical foundations, practical implementations, and real-world applications. Each section includes detailed documentation, code examples, and visual comparisons.

**Prerequisites:**
- Install required libraries: `pip install opencv-python matplotlib numpy`.
- Replace 'testImage.jpg' with your own image path if needed.
- This notebook uses Matplotlib for displaying images in subplots for side-by-side comparisons.

**Note:** All code cells assume the image is loaded in BGR format (OpenCV default). For grayscale operations, conversions are handled where necessary.

## Table of Contents
---
1. Introduction to Convolution in Image Processing
2. Mathematical Foundations of Convolution
3. Kernel Design Principles
4. Border Handling and Padding in Convolution
5. Applying Identity Kernel
6. Low-Pass Filters: Blurring Techniques
   - Custom Box Blur
   - Built-in Box Blur
   - Gaussian Blur
   - Median Blur
   - Bilateral Filter
7. High-Pass Filters: Sharpening and Edge Enhancement
   - Basic Sharpening Kernel
   - Unsharp Masking
   - Laplacian Filter for Edge Detection
   - Sobel Operators
8. Directional and Advanced Kernels
   - Emboss Filter
   - Prewitt Operator
   - Scharr Operator
   - Gabor Filters
9. Frequency Domain Filtering
   - Fourier Transform Basics
   - High-Pass and Low-Pass in Frequency Domain
10. Noise in Images: Addition and Removal
11. Performance Considerations and Optimization
12. Real-World Applications and Case Studies
13. Summary and Best Practices
---
---

## 1. Introduction to Convolution in Image Processing
---

Convolution is a fundamental operation in image processing where a kernel (small matrix) slides over the image, performing element-wise multiplication and summation to produce a filtered output.

Key applications:
- **Blurring (Low-Pass Filtering):** Reduces high-frequency components like noise or details.
- **Sharpening (High-Pass Filtering):** Enhances edges and fine details.
- **Edge Detection:** Identifies boundaries in images.
- **Feature Extraction:** Used in CNNs for machine learning.

In OpenCV, `cv2.filter2D()` is the core function for custom convolution.

## 2. Mathematical Foundations of Convolution
---

### Discrete 2D Convolution

For a 2D image $I$ and a kernel $K$ of size $(2m+1) \times (2n+1)$, the convolution at pixel $(x, y)$ is defined as:
$$
(I * K)(x, y)
=
\sum_{i=-m}^{m}
\sum_{j=-n}^{n}
I(x+i, y+j)\, K(m-i, n-j)
$$
### Key Concepts
- **1D vs 2D Convolution :** 1D convolution is used for signals (audio, time-series), while 2D convolution is used for images.  
  *Separable kernels* (e.g., Gaussian) can be decomposed into two 1D convolutions for efficiency.
- **Normalization :** Kernels are often normalized by dividing each element by the sum of all kernel values to preserve intensity.
- **Cross-Correlation vs Convolution :** OpenCV’s `filter2D()` applies **cross-correlation** (kernel is not flipped).
- **symmetric kernels :** convolution and cross-correlation are mathematically equivalent.


## 3. Kernel Design Principles
---
- **Size:** Odd dimensions (e.g., 3x3, 5x5) center easily.
- **Symmetry:** Often symmetric for isotropic effects.
- **Sum:** For low-pass, sum=1 (normalized); for high-pass, sum=0 or 1.
- **Examples:**
  - Identity: No change.
  - Box: Uniform averaging.
  - Gaussian: Weighted averaging based on distance.

## 4. Border Handling and Padding in Convolution
---
When the kernel overlaps image edges, padding is needed. OpenCV's `borderType` in `filter2D()`:
- `cv2.BORDER_CONSTANT`: Pad with constant value.
- `cv2.BORDER_REPLICATE`: Repeat edge pixels.
- `cv2.BORDER_REFLECT`: Mirror reflection.
- `cv2.BORDER_WRAP`: Wrap around.
- Default: `cv2.BORDER_DEFAULT` (reflect with no edge pixel flip).

Example: Padding affects edge artifacts in filtering.

In [ ]:
# Import all dependencies
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt
from tools.tools import LearnTools

learn_tools = LearnTools()

In [ ]:
# Find and Load the image

if os.path.exists('testImage.jpg'):
    image = cv2.imread('testImage.jpg')
    print('Image Exists')
elif not os.path.exists('testImage.jpg'):
    image_url = "https://i.ibb.co.com/BVSYcmyY/joey-kyber-GPxgi4-J82-E4-unsplash.jpg"
    pil_image = await learn_tools.get_image(img_url=image_url, padding=0)
    pil_image.save('testImage.jpg', 'JPEG')
    # or
    image = learn_tools.pil_to_cv2(pil_image=pil_image)
    # cv2.imwrite('testImage.jpg', image)
    print('Image Created')

if image is not None:
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)


    # Display the images using LearnTools
    learn_tools.show_multiple_images(
        image_plotting_data=[
            {'title': 'Original Image','image': image},
            {'title': 'Gray Image','image': gray_image}
        ]
    )
else:
    print('The image could not be loaded.')

## 5. Applying Identity Kernel
The identity kernel preserves the original image:
$$
K =
\begin{bmatrix}
0 & 0 & 0 \\
0 & 1 & 0 \\
0 & 0 & 0
\end{bmatrix}
$$
This demonstrates basic convolution without modifying the image.

What `ddepth` actually means?

`ddepth` = destination depth

It defines the data type of the output image, not the kernel.

Examples of depths:
- `cv2.CV_8U` → `uint8`
- `cv2.CV_16S` → `int16`
- `cv2.CV_32F` → `float32`
- `cv2.CV_64F` → `float64`

`ddepth` Use cases:
- `cv2.CV_32F` → Make it float
- `cv2.CV_16S` → Use signed 16-bit
- `-1` → Don’t decide. Just reuse the input type.

In [ ]:
if image is not None:
    # Step 1: Define the identity kernel
    kernel_identity = np.array(
        [
            [0, 0, 0],
            [0, 1, 0],
            [0, 0, 0]
        ]
    )

    # Step 2: Apply convolution using the identity kernel
    identity_img = cv2.filter2D(src=image, ddepth=-1, kernel=kernel_identity)

    # Display the Original Image and Identity Image using LearnTools
    learn_tools.show_multiple_images(
        image_plotting_data=[
            {'title': 'Original Image', 'image': image},
            {'title': 'Identity Filtered Image', 'image': identity_img}
        ]
    )

    # Optional: Save the resulting image to disk
    # cv2.imwrite('Identity_Image.jpg', identity_img)


## 6. Low-Pass Filters: Blurring Techniques

Low-pass filters attenuate high-frequency components, resulting in a smoother image.

### 6.1 Custom Box Blur

A box blur uses a **normalized averaging kernel**:

$$
K = \frac{1}{50}
\begin{bmatrix}
1 & 1 & 1 & 1 & 1 & 1 & 1 & 1 \\
1 & 1 & 1 & 1 & 1 & 1 & 1 & 1 \\
1 & 1 & 1 & 1 & 1 & 1 & 1 & 1 \\
1 & 1 & 1 & 1 & 1 & 1 & 1 & 1 \\
1 & 1 & 1 & 1 & 1 & 1 & 1 & 1 \\
1 & 1 & 1 & 1 & 1 & 1 & 1 & 1 \\
1 & 1 & 1 & 1 & 1 & 1 & 1 & 1 \\
1 & 1 & 1 & 1 & 1 & 1 & 1 & 1
\end{bmatrix}
\quad (8 \times 8)
$$


In [ ]:
if image is not None:
    # Step 1: Define a custom 5x5 box blur kernel
    kernel_box = np.ones((8, 8), np.float32) / 50

    # Step 2: Apply convolution with the box blur kernel
    box_blur_custom = cv2.filter2D(src=image, ddepth=-1, kernel=kernel_box)

    # Step 3: Display the original and blurred images side by side
    learn_tools.show_multiple_images(
        image_plotting_data=[
            {'title': 'Original Image', 'image': image},
            {'title': 'Custom Box Blurr Image', 'image': box_blur_custom}
        ]
    )

    # Step 4: Save the blurred image to disk
    # cv2.imwrite('custom_box_blur_image.jpg', box_blur_custom)


### 6.2 Built-in Box Blur

OpenCV's `cv2.blur()` is optimized for box filtering.

In [ ]:
if image is not None:
    # Step 1: Apply OpenCV's built-in box blur function
    box_blur_builtin = cv2.blur(src=image, ksize=(8, 8))

    # Step 2: Display the original and blurred images side by side
    learn_tools.show_multiple_images(
        image_plotting_data=[
            {'title': 'Original Image', 'image': image},
            {'title': 'Built-in Box Blur Image', 'image': box_blur_builtin}
        ]
    )

    # Optional: Save the resulting image
    # cv2.imwrite('box_blur_builtin.jpg', box_blur_builtin)


### 6.3 Gaussian Blur

Gaussian blur uses a kernel based on the **Gaussian (normal) distribution**.  
It smooths the image while preserving edges better than a simple box blur.

- **Sigma (σ)** controls the **spread of the Gaussian**:  
  - Larger σ → more blur  
  - Smaller σ → less blur

The 2D Gaussian function is defined as:

$$
G(x, y) = \frac{1}{2 \pi \sigma^2} \, e^{-\frac{x^2 + y^2}{2 \sigma^2}}
$$

- Here, \(x\) and \(y\) are the distances from the kernel center.  
- The kernel weights decrease with distance from the center, giving a **weighted average** that emphasizes nearby pixels.

**Key Notes:**
1. Gaussian blur is a **low-pass filter**, reducing high-frequency noise.  
2. Unlike a box blur, it gives **smoother, visually pleasing results** because edges are less harshly averaged.  
3. In OpenCV, you can apply it using `cv2.GaussianBlur(image, ksize, sigmaX)`.

**Parameter breakdown**
`src=image`
- Input image (grayscale or color).
- Must be a NumPy array.
- If it’s noisy, this operation reduces that noise.

`ksize=(5, 5)`
- Size of the convolution kernel.
- Must be odd and positive.
- (5,5) means each pixel is recalculated using a 5×5 neighborhood.
- Larger kernel → stronger blur → more detail loss.

`Quick intuition:`
    - (3,3) → light smoothing
    - (5,5) → moderate blur ✅ (common default)
    - (9,9) → heavy blur

`sigmaX=0`
- Standard deviation in X direction.
- 0 means OpenCV auto-computes sigma from kernel size.
- For (5,5), OpenCV picks a reasonable sigma so you don’t have to tune it.
***(Internally: `sigma ≈ 0.3*((ksize−1)*0.5 − 1) + 0.8`)***

| Kernel  | Typical Sigma |
| ------- | ------------- |
| (3,3)   | ~0.8          |
| (5,5)   | ~1.2          |
| (9,9)   | ~1.8          |
| (15,15) | ~3.0          |


In [ ]:
if image is not None:
    # Step 1: Apply Gaussian blur
    gaussian_blur = cv2.GaussianBlur(src=image, ksize=(15, 15), sigmaX=3)

    # Step 2: Display the original and blurred images side by side
    learn_tools.show_multiple_images(
        image_plotting_data=[
            {'title': 'Original Image', 'image': image},
            {'title': 'Gaussian Blur Image', 'image': gaussian_blur}
        ]
    )

    # Optional: Save the blurred image to disk
    # cv2.imwrite('gaussian_blur.jpg', gaussian_blur)


### 6.4 Median Blur
Median blur is a non-linear spatial filter that replaces each pixel value with the median of the surrounding neighborhood.Unlike Gaussian blur, it does not average values, which makes it highly effective at removing impulse (salt-and-pepper) noise while preserving edges.

**How Median Blur Works**
For each pixel:
- Collect neighboring pixel values defined by the kernel
- Sort the values
- Replace the center pixel with the median value

No weighting. No convolution. Just statistics.Mathematically, for a neighborhood `W`:
$$
I′(x,y)=median{I(i,j)∣(i,j)∈W}
$$

**Why Median Instead of Mean**
- Median rejects outliers
- Sharp intensity spikes (0 or 255) are eliminated
- Preserves edges better than linear filters

***Key Notes***
- Median blur is a non-linear filter, so it cannot be represented by convolution.
- It is highly effective for salt-and-pepper noise.
- Edges are preserved better than Gaussian blur.
- Sorting makes it computationally heavier.
- Use case : `cv2.medianBlur(image, ksize)`

**Parameter Breakdown**
`src = image`
- Input image (grayscale or color)
- Must be a NumPy array
- Best suited for impulse-noise–corrupted images

`ksize = 5`
- Size of the square neighborhood
- Must be odd and greater than 1
- Determines the number of pixels used to compute the median

***Quick Intuition***
- 3 → removes light impulse noise, minimal detail loss ✅
- 5 → strong salt-and-pepper noise suppression
- 7+ → heavy smoothing, important details start disappearing




In [ ]:
if image is not None:
    # Step 1: Apply Median blur
    # Median blur is a non-linear filter that replaces each pixel
    # with the median value of its neighborhood (k x k).
    # It is especially effective for salt-and-pepper noise.
    median_blur = cv2.medianBlur(src=image, ksize=7)

    # Step 2: Display the original and median-blurred images side by side
    learn_tools.show_multiple_images(
        image_plotting_data=[
            {'title': 'Original Image', 'image': image},
            {'title': 'Median Blur Image', 'image': median_blur}
        ]
    )

    # Optional: Save the median-blurred image to disk
    # cv2.imwrite('median_blur.jpg', median_blur)


### 6.5 Bilateral Filter
Bilateral filter is an edge-preserving, noise-reducing filter. Unlike Gaussian blur, it considers both spatial proximity and pixel intensity differences, so edges are preserved while smoothing flat regions.

**How Bilateral Filter Works**

For each pixel:
- Consider a neighborhood around the pixel.
- Compute a weighted average where weights depend on:
    - **Spatial distance**: nearby pixels contribute more.
    - **Intensity difference**: pixels with similar colors contribute more.
- Replace the pixel with the weighted average.

Mathematically, for a pixel $I(x, y)$:

$$
I_{\text{out}}(x, y) = \frac{1}{W_p} \sum_{(i, j) \in N_k} 
I(i, j) \,
\exp\Bigg(-\frac{(i - x)^2 + (j - y)^2}{2 \sigma_s^2}\Bigg) 
\exp\Bigg(-\frac{|I(i, j) - I(x, y)|^2}{2 \sigma_r^2}\Bigg)
$$

where the **normalization factor** $W_p$ ensures that the weights sum to 1:

$$
W_p = \sum_{(i, j) \in N_k} 
\exp\Bigg(-\frac{(i - x)^2 + (j - y)^2}{2 \sigma_s^2}\Bigg) 
\exp\Bigg(-\frac{|I(i, j) - I(x, y)|^2}{2 \sigma_r^2}\Bigg)
$$

**Parameters:**
- $N_k$: neighborhood of the pixel $(x, y)$  
- $\sigma_s$: spatial standard deviation (controls how much nearby pixels influence the filter)  
- $\sigma_r$: range (intensity) standard deviation (controls how much pixels with different intensities are considered)  

The bilateral filter smooths regions while preserving edges, making it very effective for noise reduction without blurring edges.


In [ ]:
if image is not None:
    # Apply bilateral filter
    # d = diameter of pixel neighborhood
    # sigmaColor = filter sigma in color space
    # sigmaSpace = filter sigma in coordinate space
    bilateral = cv2.bilateralFilter(src=image, d=15, sigmaColor=75, sigmaSpace=75)

    # Display original and filtered images side by side
    learn_tools.show_multiple_images(
        image_plotting_data=[
            {'title': 'Original Image', 'image': image},
            {'title': 'Bilateral Image', 'image': bilateral}
        ]
    )

    # Optionally save the filtered image
    # cv2.imwrite('bilateral.jpg', bilateral)
else:
    print("No image loaded.")


## 7. High-Pass Filters: Sharpening and Edge Enhancement

High-pass filters are used to **emphasize edges and fine details** in an image.  
They work by **removing low-frequency components** (smooth areas) from the original image, highlighting rapid intensity changes.

A simple and commonly used 3x3 sharpening kernel is:

$$
K = 
\begin{bmatrix}
0 & -1 & 0 \\
-1 & 5 & -1 \\
0 & -1 & 0
\end{bmatrix}
$$

**Explanation:**
- The center value `5` amplifies the current pixel.
- The `-1` values subtract contributions from the 4 immediate neighbors.
- The sum of kernel coefficients is 1, preserving overall brightness.
- This kernel enhances edges while leaving flat regions mostly unchanged.


In [ ]:
if image is not None:
    # Define a 3x3 high-pass sharpening kernel
    # Center value amplifies the pixel
    # Negative neighbors subtract surrounding intensities (edge enhancement)
    kernel_sharpen = np.array([[0, -1, 0],
                               [-1,  5, -1],
                               [0, -1, 0]])

    # Apply convolution-based sharpening
    # ddepth = -1 keeps the output image depth same as input
    sharpened = cv2.filter2D(src=image, ddepth=-1, kernel=kernel_sharpen)

    # Display original and sharpened images side by side
    learn_tools.show_multiple_images(
        image_plotting_data=[
            {'title': 'Original Image', 'image': image},
            {'title': 'Sharpen Image', 'image': sharpened}
        ]
    )

    # Optionally save the sharpened image
    # cv2.imwrite('sharpened.jpg', sharpened)
else:
    print("No image loaded.")

### 7.2 Unsharp Masking

Unsharp masking is a widely used **high-pass sharpening technique** that enhances edges while preserving overall image structure.  
Instead of applying a fixed kernel, it sharpens an image by **adding scaled high-frequency details** back to the original image.

#### Principle

1. Blur the original image to remove high-frequency components.
2. Subtract the blurred image from the original to obtain edge details.
3. Add the scaled edge details back to the original image.

This enhances edges without significantly affecting smooth regions.

#### Mathematical Formulation

Let:
- $I_{\text{original}}$ be the original image
- $I_{\text{blurred}}$ be the low-pass (blurred) image
- $\alpha$ be the sharpening strength

Then:

$$
I_{\text{sharpened}} =
I_{\text{original}} + \alpha \cdot
\left( I_{\text{original}} - I_{\text{blurred}} \right)
$$

- $(I_{\text{original}} - I_{\text{blurred}})$ acts as a **high-pass filter**
- $\alpha$ controls how strong the sharpening effect is

In [ ]:
if image is not None:
    # Apply Gaussian blur to create a low-pass version of the image
    # (0, 0) lets OpenCV compute kernel size from sigma
    # sigmaX controls the amount of smoothing
    blurred = cv2.GaussianBlur(image, (0, 0), sigmaX=3)

    # Apply unsharp masking
    # 1.5 -> weight of the original image (detail amplification)
    # -0.5 -> weight of the blurred image (detail subtraction)
    # 0 -> scalar added to the result
    unsharp_mask = cv2.addWeighted(image, 1.5, blurred, -0.5, 0)

    # Display original and unsharp-masked images side by side
    learn_tools.show_multiple_images(
        image_plotting_data=[
            {'title': 'Original Image', 'image': image},
            {'title': 'Unshaped Mask Image', 'image': unsharp_mask}
        ]
    )

    # Optionally save the sharpened image
    # cv2.imwrite('unsharp_mask.jpg', unsharp_mask)
else:
    print("No image loaded.")


### 7.3 Laplacian Filter for Edge Detection

The Laplacian filter is a **second-order derivative operator** used to highlight regions of rapid intensity change in an image.  
Unlike gradient-based filters (e.g., Sobel), the Laplacian responds to edges **in all directions equally**.

#### Principle

- Flat regions produce values close to zero.
- Sudden intensity transitions (edges) produce large positive or negative responses.
- Because it is a second derivative, the Laplacian is **highly sensitive to noise**.

For this reason, it is often applied after a **Gaussian blur**.

#### Laplacian Kernel

A commonly used 3×3 Laplacian kernel is:

$$
K =
\begin{bmatrix}
0 & 1 & 0 \\
1 & -4 & 1 \\
0 & 1 & 0
\end{bmatrix}
$$

- The negative center emphasizes intensity discontinuities.
- Positive neighboring values capture surrounding pixel changes.
- The sum of coefficients is zero, making it a **true high-pass filter**.

#### Mathematical Interpretation

The Laplacian operator approximates second-order spatial derivatives:

$$
\nabla^2 I = \frac{\partial^2 I}{\partial x^2}
           + \frac{\partial^2 I}{\partial y^2}
$$

Large values of \( \nabla^2 I \) indicate edges or fine detail.

In [ ]:
if gray_image is not None:
    # Apply Laplacian filter (second-order derivative)
    # ddepth = CV_64F allows negative values from second derivative calculation
    # ksize = 3 defines the aperture size for the Laplacian operator
    laplacian = cv2.Laplacian(gray_image, ddepth=cv2.CV_64F, ksize=3)

    # Convert result to 8-bit absolute values for visualization
    laplacian_abs = cv2.convertScaleAbs(laplacian)

    # Display grayscale original and Laplacian edge image side by side
    learn_tools.show_multiple_images(
        image_plotting_data=[
            {'title': 'Gray Original Image', 'image': gray_image},
            {'title': 'Laplacian Image', 'image': laplacian_abs}
        ]
    )

    # Optionally save the result
    # cv2.imwrite('laplacian.jpg', laplacian_abs)
else:
    print("No grayscale image loaded.")


### 7.4 Sobel Operators

Sobel operators compute the **first-order spatial derivative** of an image and are widely used for **edge detection**.  
They respond strongly to edges aligned in the **horizontal** or **vertical** direction.

#### Principle

- Measures the gradient (rate of intensity change) in the image.
- Produces two gradient images:
  - $G_x$: horizontal (vertical-edge) response
  - $G_y$: vertical (horizontal-edge) response
- Combines both to estimate edge strength and orientation.

#### Sobel Kernels

**Horizontal Gradient ($K_x$):**

$$
K_x =
\begin{bmatrix}
-1 & 0 & 1 \\
-2 & 0 & 2 \\
-1 & 0 & 1
\end{bmatrix}
$$

**Vertical Gradient ($K_y$):**

$$
K_y =
\begin{bmatrix}
-1 & -2 & -1 \\
0  &  0 &  0 \\
1  &  2 &  1
\end{bmatrix}
$$

- Coefficients emphasize intensity differences across directions.
- Larger center weights improve noise robustness compared to simple gradient filters.

In [ ]:
if gray_image is not None:
    # Compute Sobel gradient in the x-direction (vertical edges)
    # CV_64F is used to preserve negative gradient values
    # dx = 1, dy = 0 → horizontal derivative
    sobel_x = cv2.Sobel(gray_image, cv2.CV_64F, dx=1, dy=0, ksize=3)

    # Compute Sobel gradient in the y-direction (horizontal edges)
    # dx = 0, dy = 1 → vertical derivative
    sobel_y = cv2.Sobel(gray_image, cv2.CV_64F, dx=0, dy=1, ksize=3)

    # Combine x and y gradients to compute edge magnitude
    sobel_combined = cv2.magnitude(sobel_x, sobel_y)

    # Convert to 8-bit absolute values for display purposes
    sobel_combined_abs = cv2.convertScaleAbs(sobel_combined)

    # Display original grayscale image and Sobel edge map side by side
    learn_tools.show_multiple_images(
        image_plotting_data=[
            {'title': 'Gray Original Image', 'image': gray_image},
            {'title': 'Sobel Edge Image', 'image': sobel_combined_abs}
        ]
    )

    # Optionally save the result
    # cv2.imwrite('sobel.jpg', sobel_combined_abs)
else:
    print("No grayscale image loaded.")

## 8. Directional and Advanced Kernels

Directional kernels emphasize edges and textures in **specific directions**.  
They are often used for **stylized effects**, feature enhancement, or simulating lighting.

---

### 8.1 Emboss Filter

The emboss filter creates a **3D-like relief effect** by simulating directional lighting across image edges.  
It highlights intensity transitions in a specific direction while suppressing others, giving the illusion of depth.

---

#### Principle

- Enhances edges by exaggerating differences along a chosen direction.
- Shifts pixel intensities to simulate light coming from one side.
- Produces bright and dark regions resembling raised and recessed surfaces.

---

#### Emboss Kernel

A commonly used 3×3 emboss kernel is:

$$
K =
\begin{bmatrix}
-2 & -1 & 0 \\
-1 &  1 & 1 \\
 0 &  1 & 2
\end{bmatrix}
$$

- Negative values suppress pixels on one side of the edge.
- Positive values enhance pixels on the opposite side.
- The kernel direction defines the **apparent light source direction**.

In [ ]:
if image is not None:
    # Define a 3x3 emboss kernel
    # Negative values darken pixels on one side of the edge
    # Positive values brighten pixels on the opposite side
    kernel_emboss = np.array([
        [-2, -1, 0],
        [-1,  1, 1],
        [ 0,  1, 2]
    ])

    # Apply convolution to create emboss effect
    # ddepth = -1 keeps original image depth
    emboss = cv2.filter2D(src=image, ddepth=-1, kernel=kernel_emboss)

    # Display original and embossed images side by side
    learn_tools.show_multiple_images(
        image_plotting_data=[
            {'title': 'Original Image', 'image': image},
            {'title': 'Emboss Image', 'image': emboss}
        ]
    )

    # Optionally save the result
    # cv2.imwrite('emboss.jpg', emboss)
else:
    print("No image loaded.")


### 8.2 Prewitt Operator

The Prewitt operator is a simple **first-order derivative filter** used for edge detection.  
It is similar to the Sobel operator but **without weighting the center row/column**.

---

#### Principle

- Measures the gradient in an image to detect edges.
- Produces horizontal and vertical responses:
  - $G_x$: detects vertical edges (changes along x)
  - $G_y$: detects horizontal edges (changes along y)
- Less smooth than Sobel but simpler to compute.

---

#### Prewitt Kernels

**Horizontal Gradient ($K_x$):**

$$
K_x =
\begin{bmatrix}
-1 & 0 & 1 \\
-1 & 0 & 1 \\
-1 & 0 & 1
\end{bmatrix}
$$

**Vertical Gradient ($K_y$):**

$$
K_y =
\begin{bmatrix}
-1 & -1 & -1 \\
0  &  0 &  0 \\
1  &  1 &  1
\end{bmatrix}
$$

- Unlike Sobel, all rows/columns are equally weighted.
- Simpler and faster but more sensitive to noise.

---

#### Gradient Magnitude

The edge strength is computed as:

$$
G = \sqrt{G_x^2 + G_y^2}
$$

In [ ]:
if gray_image is not None:
    # Define the horizontal Prewitt kernel
    # Detects vertical edges (changes along the x-axis)
    kernel_prewitt_x = np.array([
        [-1, 0, 1],
        [-1, 0, 1],
        [-1, 0, 1]
    ], dtype=np.float32)

    # Apply convolution to extract horizontal gradient
    prewitt_x = cv2.filter2D(gray_image, ddepth=-1, kernel=kernel_prewitt_x)

    # Display grayscale original and Prewitt X edge image side by side
    learn_tools.show_multiple_images(
        image_plotting_data=[
            {'title': 'Gray Original Image', 'image': gray_image},
            {'title': 'Prewitt X Image', 'image': prewitt_x}
        ]
    )

    # Optionally save the result
    # cv2.imwrite('prewitt.jpg', prewitt_x)
else:
    print("No grayscale image loaded.")


### 8.3 Scharr Operator

The Scharr operator is an **optimized version of the Sobel operator**, designed to improve **rotation invariance** and edge detection accuracy, especially for small kernels (3×3).

---

#### Principle

- Computes the **first-order derivative** of the image, like Sobel.
- Uses **larger weights** for the center pixels to reduce rotational artifacts.
- Provides **better edge detection** for diagonal and small-scale features compared to Sobel.

---

#### Scharr Kernels

**Horizontal Gradient ($K_x$):**

$$
K_x =
\begin{bmatrix}
 3 & 0 & -3 \\
10 & 0 & -10 \\
 3 & 0 & -3
\end{bmatrix}
$$

**Vertical Gradient ($K_y$):**

$$
K_y =
\begin{bmatrix}
 3 & 10 & 3 \\
 0 &  0 & 0 \\
-3 & -10 & -3
\end{bmatrix}
$$

- The larger weights (10 vs 2 in Sobel) improve sensitivity to edges and reduce rotational bias.

---

#### Gradient Magnitude

The overall edge strength is computed as:

$$
G = \sqrt{G_x^2 + G_y^2}
$$

In [ ]:
if gray_image is not None:
    # Compute Scharr gradient in x-direction (vertical edges)
    # CV_64F preserves negative values from derivative calculation
    # dx=1, dy=0 → horizontal derivative
    scharr_x = cv2.Scharr(gray_image, ddepth=cv2.CV_64F, dx=1, dy=0)

    # Convert result to 8-bit absolute values for display
    scharr_abs = cv2.convertScaleAbs(scharr_x)

    # Display original grayscale and Scharr X edge image side by side
    learn_tools.show_multiple_images(
        image_plotting_data=[
            {'title': 'Gray Original Image', 'image': gray_image},
            {'title': 'Scharr X Image', 'image': scharr_abs}
        ]
    )

    # Optionally save the result
    # cv2.imwrite('scharr.jpg', scharr_abs)
else:
    print("No grayscale image loaded.")

### 8.4 Gabor Filters

Gabor filters are **frequency- and orientation-selective** filters widely used for **texture analysis, feature extraction, and edge detection**.  
They model the response of **visual cortical cells** and are sensitive to specific spatial frequencies and orientations.

---

#### Principle

- Combines a **Gaussian envelope** with a **sinusoidal plane wave**.
- Responds strongly to edges or textures aligned with its orientation.
- Useful for detecting **periodic patterns, textures, and directional features**.

---

#### Gabor Kernel

Mathematically, a 2D Gabor kernel is defined as:

$$
g(x, y) = \exp\Bigg(-\frac{x'^2 + \gamma^2 y'^2}{2\sigma^2}\Bigg)
          \cos\Big(2\pi \frac{x'}{\lambda} + \psi \Big)
$$

Where:

- $x' = x \cos\theta + y \sin\theta$  
- $y' = -x \sin\theta + y \cos\theta$  
- $\lambda$: wavelength of the sinusoid (controls frequency)  
- $\theta$: orientation of the filter  
- $\psi$: phase offset  
- $\sigma$: standard deviation of Gaussian envelope  
- $\gamma$: spatial aspect ratio (ellipticity)

In [ ]:
if gray_image is not None:
    # Define Gabor kernel parameters
    # ksize = size of the filter (width, height)
    # sigma = standard deviation of Gaussian envelope
    # theta = orientation of the filter (radians)
    # lambd = wavelength of the sinusoidal factor
    # gamma = spatial aspect ratio (ellipticity)
    # psi = phase offset
    gabor_kernel = cv2.getGaborKernel(
        ksize=(31, 31),
        sigma=4.0,
        theta=np.pi/4,
        lambd=10.0,
        gamma=0.5,
        psi=0
    )

    # Apply Gabor filter using convolution
    gabor_filtered = cv2.filter2D(gray_image, ddepth=cv2.CV_8UC3, kernel=gabor_kernel)

    # Display original and Gabor-filtered images side by side
    learn_tools.show_multiple_images(
        image_plotting_data=[
            {'title': 'Gray Original Image', 'image': gray_image},
            {'title': 'Gabor Filter Image', 'image': gabor_filtered}
        ]
    )

    # Optionally save the filtered result
    # cv2.imwrite('gabor.jpg', gabor_filtered)
else:
    print("No grayscale image loaded.")


## 9. Frequency Domain Filtering

Frequency domain filtering leverages the **Fourier Transform** to analyze and manipulate image components based on their frequency.  
A key principle:

> Convolution in the spatial domain is equivalent to multiplication in the frequency domain.

This allows **high-pass, low-pass, and band-pass filtering** to be implemented efficiently.

---

### 9.1 Fourier Transform Basics

The **Discrete Fourier Transform (DFT)** converts an image from the **spatial domain** to the **frequency domain**:

- Low frequencies: represent **slow variations**, smooth regions.
- High frequencies: represent **rapid changes**, edges, and fine details.

The 2D Fourier Transform of an image \( I(x, y) \) is:

$$
F(u, v) = \sum_{x=0}^{M-1} \sum_{y=0}^{N-1} I(x, y) \, e^{-j 2 \pi \left( \frac{ux}{M} + \frac{vy}{N} \right)}
$$

Where:

- \( M, N \) are the image dimensions.
- \( (u, v) \) are frequency coordinates.
- \( j = \sqrt{-1} \) is the imaginary unit.

In [ ]:
if gray_image is not None:
    # Compute the Discrete Fourier Transform (DFT)
    dft = cv2.dft(np.float32(gray_image), flags=cv2.DFT_COMPLEX_OUTPUT)

    # Shift the zero-frequency component to the center
    dft_shift = np.fft.fftshift(dft)

    # Compute magnitude spectrum for visualization
    magnitude_spectrum = 20 * np.log(cv2.magnitude(dft_shift[:, :, 0], dft_shift[:, :, 1]) + 1e-8)

    # Display original grayscale image and its magnitude spectrum using show_two_images
    learn_tools.show_multiple_images(
        image_plotting_data=[
            {'title': 'Gray Original Image', 'image': gray_image, 'cmap': 'gray'},
            {'title': 'Magnitude Spectrum', 'image': magnitude_spectrum, 'cmap': 'gray'},
        ]
    )


else:
    print("No grayscale image loaded.")


### 9.2 High-Pass and Low-Pass in Frequency Domain

Frequency domain filtering allows selective enhancement or suppression of certain frequency components:

- **Low-pass filter (LPF):** allows low frequencies to pass and **blocks high frequencies** → smooths the image.
- **High-pass filter (HPF):** allows high frequencies to pass and **blocks low frequencies** → emphasizes edges and fine details.

---

#### Ideal Low-Pass Filter

- Masks all frequencies outside a circular region of radius `D0` in the frequency domain.
- Let `F(u, v)` be the DFT of the image. The filtered spectrum `G(u, v)`:

$$
G(u, v) =
\begin{cases}
F(u, v), & \text{if } D(u, v) \le D_0 \\
0, & \text{if } D(u, v) > D_0
\end{cases}
$$

Where:

- \( D(u, v) = \sqrt{(u - M/2)^2 + (v - N/2)^2} \) is the distance from the center frequency.
- \( D_0 \) is the cutoff frequency.

In [ ]:
if gray_image is not None:
    # Get image dimensions
    rows, cols = gray_image.shape
    crow, ccol = rows // 2, cols // 2  # center

    # Create a circular mask for low-pass filtering
    # All frequencies outside the central square region are set to 0
    mask = np.zeros((rows, cols, 2), np.uint8)
    mask[crow-30:crow+30, ccol-30:ccol+30] = 1  # square low-pass region

    # Apply the mask to the shifted DFT
    fshift = dft_shift * mask

    # Shift back and compute inverse DFT
    f_ishift = np.fft.ifftshift(fshift)
    img_back = cv2.idft(f_ishift)

    # Compute magnitude to get the filtered image
    img_back = cv2.magnitude(img_back[:, :, 0], img_back[:, :, 1])

    # Display original and low-pass filtered images side by side
    learn_tools.show_multiple_images(
        image_plotting_data=[
            {'title': 'Gray Original Image', 'image': gray_image},
            {'title': 'Low-Pass Filtered Image', 'image': img_back}
        ]
    )

    # Optionally save the filtered image
    # cv2.imwrite('freq_lowpass.jpg', img_back)
else:
    print("No grayscale image loaded.")


## 10. Noise in Images: Addition and Removal

Noise is an unwanted variation of intensity in images.  
Common types include Gaussian, salt-and-pepper, and speckle noise.  

This section demonstrates:

1. Adding **Gaussian noise** to an image.
2. Removing noise using **spatial filters** like Gaussian and bilateral filters.

---

### 10.1 Adding Gaussian Noise

Gaussian noise has a **normal distribution** with mean μ and standard deviation σ.  

- Pixels are randomly perturbed:  

$$
I_{noisy}(x, y) = I(x, y) + n(x, y), \quad n(x, y) \sim \mathcal{N}(\mu, \sigma^2)
$$

- Parameters `mean` and `sigma` control the intensity of the noise.

In [ ]:
if image is not None:
    mean = 0
    var = 10
    sigma = var ** 0.5
    gaussian_noise = np.random.normal(mean, sigma, image.shape)
    noisy_image = np.clip(image + gaussian_noise, 0, 255).astype(np.uint8)

    denoised_gaussian = cv2.GaussianBlur(noisy_image, (5, 5), 0)
    denoised_median = cv2.medianBlur(noisy_image, 5)


    # Display original and low-pass filtered images side by side
    learn_tools.show_multiple_images(
        image_plotting_data=[
            {'title': 'Original Image', 'image': image},
            {'title': 'Noisy Image', 'image': noisy_image},
            {'title': 'DeNoised Gaussian Image', 'image': denoised_gaussian},
            {'title': 'DeNoised Median Image', 'image': denoised_median}
        ]
    )

    # cv2.imwrite('noisy.jpg', noisy_image)
    # cv2.imwrite('denoised_gaussian.jpg', denoised_gaussian)


else:
    print("No image loaded.")

## 11. Performance Considerations and Optimization

- **Separable filters for speed:** Use filters like Gaussian that can be broken into 1D operations.
- **Integral images for box filters:** Enables fast computation of sum over regions.
- **GPU acceleration with OpenCV CUDA:** Leverage GPU for heavy image processing tasks.
- **Benchmarking tip:** Larger kernels are slower; prefer built-in optimized functions whenever possible.

### ✅ Notes:
- `cv2.filter2D` applies a custom kernel; slower for large kernels.
- `cv2.blur` is optimized; usually faster for standard box filters.
- You can replace `(15,15)` with any kernel size to benchmark performance differences.

In [ ]:
import cv2
import numpy as np
import time

# Make sure 'image' is loaded, e.g., image = cv2.imread("path_to_image.jpg")
if image is not None:
    # Custom filter using filter2D
    start = time.time()
    kernel = np.ones((15, 15), np.float32) / 225
    cv2.filter2D(image, -1, kernel)
    custom_time = time.time() - start

    # Built-in blur function
    start = time.time()
    cv2.blur(image, (15, 15))
    builtin_time = time.time() - start

    # Print performance comparison
    print(f'Custom Filter Time: {custom_time:.4f}s')
    print(f'Built-in Blur Time: {builtin_time:.4f}s')
